In [1]:
##import libraries
import torch
import torch.nn as nn
from torchvision import datasets,transforms,models
from torch.utils.data import DataLoader
import os
import copy
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchvision.models import resnet50, ResNet50_Weights
import ssl
import torch.optim as optim

In [2]:
## extracting contents of Data.zip
import zipfile
import os

zip_path = 'Data.zip'

#destination of extracted files
extracts = "Data"

os.makedirs(extracts,exist_ok=True)

with zipfile.ZipFile(zip_path,'r') as zip_ref:
  zip_ref.extractall(extracts)




In [3]:
import os

folder_path = "Sets/Data"

# List everything in the folder
contents = os.listdir(folder_path)

print("Contents of folder:")
for item in contents:
    print(item)


Contents of folder:
valid
test
train


In [11]:
## data transformations
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2,contrast=0.2,saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])
    ])
valid_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.228,0.224,0.225])
])
test_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.228,0.224,0.225])
])

In [9]:
##defining the model class
class ResNetLungCancer(nn.Module):
  def __init__(self,num_classes,use_pretrained=True):
    super(ResNetLungCancer,self).__init__()
    if use_pretrained:
      weights = ResNet50_Weights.IMAGENET1K_V1
    else:
      weights = None
    self.resnet = resnet50(weights=weights)
    num_ftrs = self.resnet.fc.in_features
    self.resnet.fc = nn.Identity()
    self.fc = nn.Sequential(
        nn.Linear(num_ftrs,256),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(256,num_classes)
    )

  def forward(self,x):
    x = self.resnet(x)
    return self.fc(x)

In [6]:
# train function
def train_model(model, train_loader, valid_loader, criterion, optimizer, scheduler, num_epochs=50, device='cuda'):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()
                dataloader = train_loader
            else:
                model.eval()
                dataloader = valid_loader

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloader.dataset)
            epoch_acc = running_corrects.double() / len(dataloader.dataset)

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            if phase == 'valid':
                scheduler.step(epoch_acc)
                current_lr = optimizer.param_groups[0]['lr']
                print(f'Learning rate: {current_lr}')
                if epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_model_wts = copy.deepcopy(model.state_dict())

        print()

    print(f'Best val Acc: {best_acc:.4f}')
    model.load_state_dict(best_model_wts)
    return model

In [7]:
# eval the model
def evaluate_model(model, test_loader, device='cuda'):
    model.eval()
    running_corrects = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            running_corrects += torch.sum(preds == labels.data)

    test_acc = running_corrects.double() / len(test_loader.dataset)
    print(f'Test Acc: {test_acc:.4f}')

In [14]:
!pip install onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 99.9 MB/s eta 0:00:00


In [15]:
if __name__ == '__main__':
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  print(f"Using device: {device}")

  #data
  data_dir = "Sets/Data"


  train_dataset = datasets.ImageFolder(os.path.join(data_dir, 'train'), transform=train_transforms)
  valid_dataset = datasets.ImageFolder(os.path.join(data_dir, 'valid'), transform=valid_transforms)
  test_dataset = datasets.ImageFolder(os.path.join(data_dir, 'test'), transform=test_transforms)

  batch_size=32

  train_loader = DataLoader(train_dataset,batch_size=batch_size,shuffle=True,num_workers=4)
  valid_loader = DataLoader(valid_dataset,batch_size=batch_size,shuffle=False,num_workers=4)
  test_loader = DataLoader(test_dataset,batch_size=batch_size,shuffle=False,num_workers=4)

  print(f"Number of training images: {len(train_dataset)}")
  print(f"Number of validation images: {len(valid_dataset)}")
  print(f"Number of testing images: {len(test_dataset)}")

  #intializing model,loss and optimizer
  num_classes = len(train_dataset.classes)
  model = ResNetLungCancer(num_classes)
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model = model.to(device)


  criterion = nn.CrossEntropyLoss()

  pretrained_params = list(model.resnet.parameters())
  new_params  = list(model.fc.parameters())

  optimizer = optim.Adam([
       {'params':pretrained_params,'lr':1e-5},
      {'params':new_params,'lr':1e-4}
         ],weight_decay=1e-6)

  scheduler = ReduceLROnPlateau(optimizer,patience=7,mode='max',factor=0.5)

  # train the model
  trained_model = train_model(model, train_loader, valid_loader, criterion, optimizer, scheduler, num_epochs=50, device=device)

    # eval the model
  evaluate_model(trained_model, test_loader, device=device)

    # save the model weights
  torch.save(trained_model.state_dict(), 'lung_cancer_detection_model.pth')

    # save the model in ONNX format
  dummy_input = torch.randn(1, 3, 224, 224).to(device)
  torch.onnx.export(trained_model, dummy_input, "lung_cancer_detection_model.onnx", input_names=['input'], output_names=['output'])

  print("Training completed. Model saved.")

Using device: cuda:0
Number of training images: 613
Number of validation images: 72
Number of testing images: 315


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch 0/49
----------
train Loss: 1.3373 Acc: 0.3409
valid Loss: 1.3093 Acc: 0.3333
Learning rate: 1e-05

Epoch 1/49
----------
train Loss: 1.1875 Acc: 0.4812
valid Loss: 1.1224 Acc: 0.4861
Learning rate: 1e-05

Epoch 2/49
----------
train Loss: 1.0307 Acc: 0.5383
valid Loss: 0.9753 Acc: 0.5139
Learning rate: 1e-05

Epoch 3/49
----------
train Loss: 0.9385 Acc: 0.5710
valid Loss: 0.9479 Acc: 0.5139
Learning rate: 1e-05

Epoch 4/49
----------
train Loss: 0.9361 Acc: 0.5579
valid Loss: 0.9354 Acc: 0.5694
Learning rate: 1e-05

Epoch 5/49
----------
train Loss: 0.8883 Acc: 0.5840
valid Loss: 0.9001 Acc: 0.5694
Learning rate: 1e-05

Epoch 6/49
----------
train Loss: 0.8642 Acc: 0.5905
valid Loss: 0.8911 Acc: 0.5417
Learning rate: 1e-05

Epoch 7/49
----------
train Loss: 0.8090 Acc: 0.6427
valid Loss: 0.8911 Acc: 0.6111
Learning rate: 1e-05

Epoch 8/49
----------
train Loss: 0.7707 Acc: 0.6721
valid Loss: 0.8374 Acc: 0.6250
Learning rate: 1e-05

Epoch 9/49
----------
train Loss: 0.7411 Acc: 

/tmp/ipython-input-724529166.py:53: DeprecationWarning: You are using the legacy TorchScript-based ONNX export. Starting in PyTorch 2.9, the new torch.export-based ONNX exporter will be the default. To switch now, set dynamo=True in torch.onnx.export. This new exporter supports features like exporting LLMs with DynamicCache. We encourage you to try it and share feedback to help improve the experience. Learn more about the new export logic: https://pytorch.org/docs/stable/onnx_dynamo.html. For exporting control flow: https://pytorch.org/tutorials/beginner/onnx/export_control_flow_model_to_onnx_tutorial.html.
  torch.onnx.export(trained_model, dummy_input, "lung_cancer_detection_model.onnx", input_names=['input'], output_names=['output'])


Training completed. Model saved.


In [20]:
import torch
from PIL import Image
from torchvision import transforms
#from architecture import ResNetLungCancer

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = ResNetLungCancer(num_classes=4)
model.load_state_dict(torch.load('lung_cancer_detection_model.pth', map_location=device))
model = model.to(device)
model.eval()

preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# image from local file
image_path = "Sets/Data/test/adenocarcinoma/000114.png"
image = Image.open(image_path).convert('RGB')

# preprocess the image
input_tensor = preprocess(image).unsqueeze(0).to(device)  # add batch dimension and move to device

# get model predictions
with torch.no_grad():
    output = model(input_tensor)

predicted_class = torch.argmax(output, dim=1).item()

class_names = ['Adenocarcinoma', 'Large Cell Carcinoma', 'Normal', 'Squamous Cell Carcinoma']

print(f"Predicted class: {class_names[predicted_class]}")

Predicted class: Adenocarcinoma


In [22]:
import torch
from PIL import Image
from torchvision import transforms
#from architecture import ResNetLungCancer
import gradio as gr

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = ResNetLungCancer(num_classes=4)
model.load_state_dict(torch.load('lung_cancer_detection_model.pth', map_location=device))
model = model.to(device)
model.eval()

preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

class_names = ['Adenocarcinoma', 'Large Cell Carcinoma', 'Normal', 'Squamous Cell Carcinoma']

def predict(image):
    image = Image.fromarray(image.astype('uint8'), 'RGB')
    input_tensor = preprocess(image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(input_tensor)

    predicted_class = torch.argmax(output, dim=1).item()
    return class_names[predicted_class]

iface = gr.Interface(
    fn=predict,
    inputs=gr.Image(),
    outputs=gr.Label(num_top_classes=1),
    examples=[
        ["Sets/Data/test/large.cell.carcinoma/000108.png"],
        ["Sets/Data/test/normal/7 - Copy (3).png"]
    ]
)

iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ee4a768f64764a0b33.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
